In [1]:
import pandas as pd
import numpy as np
import math
import os
import pickle
import joblib
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, SGD
from sktime.classification.deep_learning.cnn import CNNClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from keras.losses import get
import tensorflow as tf
from sktime.utils import mlflow_sktime

## Import custom functions from 'pre_processing.py' ##
try:
    import importlib
    importlib.reload(pp)
except NameError: # It hasn't been imported yet
    import pre_processing as pp
    
## Import custom functions from 'CNN.py' ##
try:
    import importlib
    importlib.reload(cnn)
except NameError: # It hasn't been imported yet
    import CNN as cnn

fnirs_data = r'C:\Users\61407\mental-health-modelling\data\fNIRS\bandpass_filtered_whole_data'

## 0. Data restructuring ##
- Split the data into task blocks (16 blocks per subject, each block consists of data from one level of task difficulty), as they are independent time series (and we don't want overlap between task blocks)
- Further split the 80 second task blocks into "chunks" of 10 seconds each, to increase number of samples (8 chunks x 16 blocks = 128 samples per subject). These chunks of data are what will eventually be fed into the models

***Data format required for multivariate time series classification using sktime:***
1. Pandas dataframe (number of chunks x number of channels)
2. Within each cell of the pandas dataframe needs to be a pandas series (the time series within a chunk, for a given sensor measure)

In [2]:
structure_suffix = '20s_chunks_10s_overlap_prescaled' # suffix to add at end of model name, to indicate how data was restructure

In [31]:
chunk_seconds = 20 #how long you want each chunk to be in seconds
overlap_seconds = 10 #how many seconds of overlap you want across chunks

# Upload raw data from repository and scale on upload #
all_data = pp.upload_all_subjects(fnirs_data, scaled=True)

# First split data into task blocks by assigning a block label
all_grouped_dataframes, all_block_change_idx = pp.detect_block_changes(all_data, label='difficulty')
block_labelled_data = pd.concat(all_grouped_dataframes)

# Make sure that every block is the same size, otherwise the following line will raise an error
block_size = int(block_labelled_data.groupby(block_labelled_data.index.get_level_values(0)).size().unique())

# Split data from task blocks into 10 second chunks by assigning a chunk label
chunk_labelled_data = block_labelled_data.groupby(block_labelled_data.index.get_level_values(0)).apply(pp.sliding_window, chunk_seconds, overlap_seconds)

# Do some index restructuring for aesthetics - this is a messy code that should be optimized
chunk_labelled_data = chunk_labelled_data.droplevel(0).reset_index().set_index(['chunk','block','subject','sample'])
unique_chunk = np.array([[i+1]*math.floor(chunk_seconds*5.2) for i, (x) in enumerate(chunk_labelled_data.droplevel([2,3]).index.unique())]).flatten()
chunk_labelled_data['unique_chunk_ID'] = unique_chunk
chunk_labelled_data = chunk_labelled_data.reset_index().set_index(['unique_chunk_ID','block','chunk','subject','sample'])

In [38]:
# Define a function to convert time series representation over multiple rows of a pandas into a single array
def nested_series(group):
    return pd.Series([pd.Series(group[col].to_numpy()) for col in group.columns if col != 'difficulty'])

# Define a function to retrieve unique column values (for label extraction)
def get_unique_column_values(group, column, dtype):
    return dtype(group[column].unique())

# Convert pandas time series into a pandas of nested time series arrays
nested_data = chunk_labelled_data.groupby(chunk_labelled_data.index.get_level_values('unique_chunk_ID')).apply(nested_series)
nested_data.columns = chunk_labelled_data.drop('difficulty', axis=1).columns

# Get indexes for each chunk
chunked_indexes = [chunk_labelled_data[chunk_labelled_data.index.get_level_values('unique_chunk_ID') == x].index[0][:-1] for x in nested_data.index]
chunked_multiindex = pd.MultiIndex.from_tuples(chunked_indexes)
nested_data.index = chunked_multiindex
nested_data.index.names = ['unique_chunk_ID','block','chunk','subject']

# Get labels for each chunk
chunked_labels = chunk_labelled_data.groupby(chunk_labelled_data.index.get_level_values(0)).apply(get_unique_column_values, 'difficulty', int)
chunked_labels.index = nested_data.index

## 1. Define paragidms and data partitioning ##
***Generic model:*** The first paradigm is to try and build a "one size fits all" model that is trained on many subjects, and can generalize to new subject data. This would allow for a pre-trained model to be deployed with the software, and there will be **no model training required at the user end**. However, this would the software to cognitive workload-related tasks only, and have **limited flexibility in its applications**. Application flexibility could be increased by deploying multiple pre-trained models for different tasks, but this would require extensive and continuous data collection.
- *Data partitioning for model training will involve randomly splitting the 68 subjects into 51 training (75%) and 17 testing (25%) subjects. We will perform this split in 4 different "buckets", where every bucket will have different test subjects from each other (random selection of 17 test subjects without replacement for each bucket).*

***Subject-specific models:*** The second paradigm is to build a single model for each subject, where only a model architecture will be deployed with the software, and **model training will be required at the user end for optimiation**. This approach would **increase the flexibility of its applications beyond cognitive workload-related tasks**, where a user could train the model architecture on specific tasks that are relevant to their daily lives. This approach would probably **require a cloud interface on the developer end to accept model training requests from users**, as model training may require high computational capacity that users may not have on their local device.
- *Data partitioning for model training will involve splitting each subject's data into 12 training and 4 test blocks, where there will be one block from each level of task difficulty (4 difficulties) in the test blocks. This results in 96 training and 32 testing "chunks".*

**1.1 Partition the raw data for generic model building**
- Here we will only extract the indexes of partitions for each bucket, to avoid having to write the data partitions into memory (and will use these indexes and reference the entire data set at later steps)

In [39]:
generic_suffix = '51train_17test' # suffix to add at end of model name, to indicate how generic models were partitioned

In [40]:
# Define the desired number of test subjects in each bucket
test_size = 17

# Total number of subjects
n_subjects = all_data.index.get_level_values(0).unique().shape[0]

# Run random partitioning to get test bucket index positions
seed = 42 # for reproducibility
test_buckets_positions = pp.partition_subject_indexes(test_size, n_subjects, seed)

# Get the actual index names of test buckets based on dataframe
test_buckets = [all_data.index.get_level_values(0).unique()[x] for x in test_buckets_positions]

# Get the train buckets
train_buckets = [all_data[~all_data.index.get_level_values(0).isin(x)].index.get_level_values(0).unique() for x in test_buckets]

4 buckets
68 unique indexes
Repeated indexes: (array([], dtype=int64),)


**1.2 Partition the raw data for subject-specific model building**
- Here we will only define the test indexes, and use a boolean operation to partition into train/test in later steps

In [41]:
subject_suffix = '1test_per_difficulty' # suffix to add at end of model name, to indicate how subject models were partitioned

In [42]:
# Create groupby to get the block numbers for each task difficulty, for each subject
groups = chunk_labelled_data.groupby([chunk_labelled_data.index.get_level_values('subject'),
                                     'difficulty',
                                     ]
                                    )

# Randomly sample one block from each groupby group {subject/difficulty : block number}
test_idx = {name:np.random.choice(indices)[1] for name, indices in groups.groups.items()}

## 2. Feature engineering ##

**2.1 For these 1D convolutional neural networks, the network will automatically extract features from the data**

**2.2 Label engineering**
- Decide whether we want to predict all labels, a subset of labels (e.g. level 0 vs. level 2), or combined labels (e.g. "low" difficulty levels 0 + 1 vs. "high" difficulty levels 2 + 3).
- Each type of label engineering will have different use cases

In [43]:
label_suffix = 'low_vs_high' # 0 = low (level 0 or 1), 1 = high (level 2 or 3)

# Get labels for each chunk
chunked_labels = chunk_labelled_data.groupby(chunk_labelled_data.index.get_level_values(0)).apply(get_unique_column_values, 'difficulty', int)
chunked_labels.index = nested_data.index

# Apply label engineering
chunked_labels = chunked_labels.replace({1:0, 2:1, 3:1})
# chunked_labels = chunked_labels[chunked_labels.isin([0,2])]
# nested_data = nested_data[nested_data.index.isin(chunked_labels.index)]

## 3. Define CNN classifier, training parameters, and hyperparameter search space ##
- Data has already been scaled upon data uploading  (on a subject-by-subject basis).
- Given that there are only 8 features, and because the CNN performs automatic feature extraction, no dimensionality reduction will be performed.
- We will use the pre-built sktime CNN classifier network architecture that has been proven to be effective across many data types (inculuding neural data). The architecture itself has been tested, but these networks are not pre-trained.

In [44]:
# Define network training parameters
n_epochs = 50

# Define callback options - annoyingly, there is no in-built option to monitor validation loss/accuracy (for the sktime classifier, specifically) 
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-6)

# Hyperparameter grid
n_iter = 15 #number of different combinations within the grid you want to try
param_dist_generic = {"kernel_size": list(range(int(1*5.2), int(5*5.2))), # converting seconds to samples (5.2Hz sampling rate)
                      "avg_pool_size": list(range(2,4)),
                      "n_conv_layers": list(range(2,4)),
                      "loss":['mean_squared_error', 'categorical_crossentropy'],
                      "batch_size":[8,32,128],
                      "learning_rate":[0.1, 0.01, 0.001],
                      "activation":['sigmoid','softmax','relu','tanh'],
                      }

param_dist_subject = {"kernel_size": list(range(int(1*5.2), int(5*5.2))), # converting seconds to samples (5.2Hz sampling rate)
                      "avg_pool_size": list(range(2,4)),
                      "n_conv_layers": list(range(2,4)),
                      "loss":['mean_squared_error', 'categorical_crossentropy'],
                      "batch_size":[int(x/2) for x in [8,32,128]],
                      "learning_rate":[0.1, 0.01, 0.001],
                      "activation":['sigmoid','softmax','relu','tanh'],
                      }

In [45]:
def create_manual_random_search(param_dist, n_iter):
    np.random.seed(42)

    # Get possible indices for each hyperparameter
    limits = {key:len(value) for key, value in param_dist.items()}

    # Generate n_iter random combinations for each element in the list
    random_combinations = []
    tested_kernels = []
    for i in range(n_iter):
        combination = []
        for pos, (param, limit) in enumerate(limits.items()):
            # make sure that as many kernel sizes are tested in the search
            if param == 'kernel_size':
                pos_idx = int(np.random.choice([x for x in range(limit) if x not in tested_kernels], 1))
                tested_kernels.append(pos_idx)
            else:
                pos_idx = int(np.random.choice(limit, 1))
            combination.append(param_dist[param][pos_idx])
        random_combinations.append(combination)
    
    param_dicts = [{list(param_dist.keys())[pos]: value  for pos, (value) in enumerate(x)} for x in random_combinations]
    
    return param_dicts

## 4. Train generic models ##

In [46]:
# Dirctory of where you want to write/read these models
generic_dir = r'C:\Users\61407\mental-health-modelling\fNIRS Mental Workload\final_models\generic'

for bucket, (test, train) in enumerate(zip(test_buckets, train_buckets)):
    print(f'Bucket {bucket}:\n{"*"*120}')
    
    # Name of the model to save
    fname = f'generic_1D_CNN_bucket{bucket}_{structure_suffix}_{generic_suffix}_{label_suffix}.pkl'
    
    # Check if model already exists
    if fname in os.listdir(generic_dir):
        print(f'Already exists!\n{"*"*120}\n')
    else:
    
        # Data partitioning
        print('partitioning...')
        X_train, X_test = nested_data[nested_data.index.get_level_values('subject').isin(train)], nested_data[nested_data.index.get_level_values('subject').isin(test)]
        y_train, y_test = chunked_labels[chunked_labels.index.get_level_values('subject').isin(train)], chunked_labels[chunked_labels.index.get_level_values('subject').isin(test)]
        print('...complete!\n')
        
        # Create random search space
        random_combinations = create_manual_random_search(param_dist_generic, n_iter)
        
        # Iterate through search space
        search_CV = {}
        for n, (params) in enumerate(random_combinations):
            
            # Make sure that number of filter sizes matches the number of convolutional layers specified
            filter_sizes = [6*(2**(x)) for x in range(params['n_conv_layers'])] # make filter sizes multiples of 6, increasing in size with each layer
            
            # Create CNN instance
            clf_CNN = CNNClassifier(n_epochs=n_epochs, verbose=False, random_state=42,
                                    filter_sizes=filter_sizes, # the network is optimized for the default filter sizes apparently ([6,12] for 2 convolutional layers)
                                    callbacks=[reduce_lr, early_stopping]
                                    )
            clf_CNN.set_params(**params) # setting the hyperparameters to test
            
            # Fit and cross-validate
            try: #sometimes the fit_predict method fails due to no learning/early stopping
                y_pred_val = clf_CNN.fit_predict(X_train, y_train, cv=3, change_state=True)
            except ValueError as e:
                print(f'fit_predict() method failed due to " {e} " on iteration_{n}')
                continue
            correct_predictions = np.sum(y_pred_val == y_train)
            val_accuracy = correct_predictions / y_pred_val.shape[0]
            print(f'Avg. validation accuracy: {val_accuracy}')
            
            # For whatever reason, the reduce_lr callback can't be serialized, so it has to be manually removed after fitting
            clf_CNN.callbacks = [early_stopping] #early stopping seems to be fine
            
            # Save results from current hyperparameter settings
            search_CV[n] = {'ytrain':y_train,
                            'ypred_val':y_pred_val,
                            'val_accuracy':val_accuracy,
                            'cnn':clf_CNN}
        
        # Print the best hyperparameters for the current pipeline
        val_df = pd.DataFrame({it : x['val_accuracy'] for it, x in search_CV.items()}.values(),
                              {it : x['val_accuracy'] for it, x in search_CV.items()}.keys()
                              )
        best_iteration = val_df.idxmax()[0]
        print("Best hyperparameters:\n", random_combinations[best_iteration])

        # Get predictions on test data using the best hyperparameter settings
        best_estimator = search_CV[best_iteration]['cnn']          
              
        predictions = {}
        for sub in y_test.index.get_level_values('subject').unique():
            X_sub = X_test[X_test.index.get_level_values('subject') == sub]
            y_sub = y_test[y_test.index.get_level_values('subject') ==  sub]
            sub_predictions = best_estimator.predict(X_sub)
            predictions[sub] = {'ytest':y_sub, 'ypred_test':sub_predictions}

        # Score test predictions
        test_score = round(best_estimator.score(X_test, y_test),3)
        print(f'Test accuracy: {test_score}')
        
        # Combine results into dictionary
        bucket_results = {'predictions':predictions,
                          'test_score':test_score,
                          'scv':search_CV}
        
        # Save bucket results
        with open(os.path.join(generic_dir, fname), 'wb') as file:
            joblib.dump(bucket_results, file)
        print(f'{"*"*120}\n')

Bucket 0:
************************************************************************************************************************
partitioning...
...complete!



C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


15/15 [==============================] - 0s 3ms/step


C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


15/15 [==============================] - 0s 2ms/step


...

238/238 [==============================] - 0s 1ms/step


C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


238/238 [==============================] - 0s 1ms/step


C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


238/238 [==============================] - 0s 1ms/step
Avg. validation accuracy: 0.5
Best hyperparameters:
 {'kernel_size': 9, 'avg_pool_size': 2, 'n_conv_layers': 3, 'loss': 'categorical_crossentropy', 'batch_size': 32, 'learning_rate': 0.01, 'activation': 'sigmoid'}
60/60 [==============================] - 0s 2ms/step
Test accuracy: 0.512
************************************************************************************************************************



## 5. Same as 4. but for subject-by-subject paradigm ##

In [47]:
# Dirctory of where you want to write/read these models
subject_dir = r'C:\Users\61407\mental-health-modelling\fNIRS Mental Workload\final_models\subject'

for sub in nested_data.index.get_level_values('subject').unique():
    print(f'{sub}:\n{"*"*120}')
    fname = f'{sub}_1D_CNN_{structure_suffix}_{subject_suffix}_{label_suffix}.pkl'

    # Check if model already exists
    if fname in os.listdir(subject_dir):
        print(f'Already exists!\n{"*"*120}\n')
    else:
        
        # Data partitioning
        print('partitioning...')
        X_test = nested_data[(nested_data.index.get_level_values('subject') == sub) &
                             (nested_data.index.get_level_values('block').isin([value for key, value in test_idx.items() if key[0] == sub]))
                             ]
        y_test = chunked_labels[(chunked_labels.index.get_level_values('subject') == sub) &
                                (chunked_labels.index.get_level_values('block').isin([value for key, value in test_idx.items() if key[0] == sub]))
                               ]
        X_train = nested_data[((nested_data.index.get_level_values('subject') == sub)) &
                              (~nested_data.index.isin(X_test.index))]
        y_train = chunked_labels[((chunked_labels.index.get_level_values('subject') == sub)) &
                                (~chunked_labels.index.isin(y_test.index))]
        print('...complete!\n')

        # Create random search space
        random_combinations = create_manual_random_search(param_dist_subject, n_iter)
        
        # Iterate through search space
        search_CV = {}
        for n, (params) in enumerate(random_combinations):
            
            # Make sure that number of filter sizes matches the number of convolutional layers specified
            filter_sizes = [6*(2**(x)) for x in range(params['n_conv_layers'])] # make filter sizes multiples of 6, increasing in size with each layer
            
            # Create CNN instance
            clf_CNN = CNNClassifier(n_epochs=n_epochs, verbose=False, random_state=42,
                                    filter_sizes=filter_sizes, # the network is optimized for the default filter sizes apparently ([6,12] for 2 convolutional layers)
                                    callbacks=[reduce_lr, early_stopping]
                                    )
            clf_CNN.set_params(**params) # setting the hyperparameters to test
            
            # Fit and cross-validate
            try: #sometimes the fit_predict method fails due to no learning/early stopping
                y_pred_val = clf_CNN.fit_predict(X_train, y_train, cv=3, change_state=True)
            except ValueError as e:
                print(f'fit_predict() method failed due to " {e} " on iteration_{n}')
                continue
            correct_predictions = np.sum(y_pred_val == y_train)
            val_accuracy = correct_predictions / y_pred_val.shape[0]
            print(f'Avg. validation accuracy: {val_accuracy}')
            
            # For whatever reason, the reduce_lr callback can't be serialized, so it has to be manually removed after fitting
            clf_CNN.callbacks = [early_stopping] #early stopping seems to be fine
            
            # Save results from current hyperparameter settings
            search_CV[n] = {'ytrain':y_train,
                            'ypred_val':y_pred_val,
                            'val_accuracy':val_accuracy,
                            'cnn':clf_CNN}

        # Print the best hyperparameters for the current pipeline
        val_df = pd.DataFrame({it : x['val_accuracy'] for it, x in search_CV.items()}.values(),
                              {it : x['val_accuracy'] for it, x in search_CV.items()}.keys()
                              )
        best_iteration = val_df.idxmax()[0]
        print("Best hyperparameters:\n", random_combinations[best_iteration])

        # Get predictions on test data using the best hyperparameter settings
        best_estimator = search_CV[best_iteration]['cnn'] 
        y_test_pred = best_estimator.predict(X_test)
        test_score = best_estimator.score(X_test, y_test)
        print(f'Test accuracy: {test_score}')      
        
        # Save best estimator test predictions for later
        search_CV['best_test'] = {'y_true':y_test, 'y_pred':y_test_pred, 'accuracy':test_score}
        
        # Save subject results
        with open(os.path.join(subject_dir, fname), 'wb') as file:
            joblib.dump(search_CV, file)
        print(f'{"*"*120}\n')

sub_1:
************************************************************************************************************************
partitioning...
...complete!



C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1/1 [==============================] - 0s 84ms/step


C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1/1 [==============================] - 0s 72ms/step


...

7/7 [==============================] - 0s 2ms/step


C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


7/7 [==============================] - 0s 2ms/step


C:\Users\61407\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


7/7 [==============================] - 0s 2ms/step
Avg. validation accuracy: 0.9166666666666666
Best hyperparameters:
 {'kernel_size': 12, 'avg_pool_size': 3, 'n_conv_layers': 2, 'loss': 'mean_squared_error', 'batch_size': 64, 'learning_rate': 0.001, 'activation': 'tanh'}
1/1 [==============================] - 0s 19ms/step
Test accuracy: 0.5
************************************************************************************************************************

